# Testing performance scores reported for the ISIC2016 dataset

In [1]:
from mlscorecheck.check.bundles.skinlesion import check_isic2016

In [2]:
scores = {'acc': 0.7916, 'sens': 0.2933, 'spec': 0.9145}

In [3]:
results = check_isic2016(scores=scores, eps=1e-4)
results['inconsistency']

2023-11-03 08:52:30,546:INFO:Use this function if the scores originate from the tp and tn statistics calculated on one test set with no aggregation of any kind.
2023-11-03 08:52:30,547:INFO:calling the score check with scores {'acc': 0.7916, 'sens': 0.2933, 'spec': 0.9145}, uncertainty 0.0001, p 75 and n 304
2023-11-03 08:52:30,549:INFO:checking the scores {'acc': 0.7916, 'sens': 0.2933, 'spec': 0.9145}
2023-11-03 08:52:30,551:INFO:evaluating the tp and tn solution for acc and sens
2023-11-03 08:52:30,552:INFO:intervals before: (0, 75), (0, 304)
2023-11-03 08:52:30,554:INFO:the tp solutions: (21.98985, 22.00515)
2023-11-03 08:52:30,555:INFO:the tn solutions: (277.97259199999996, 278.06520800000004)
2023-11-03 08:52:30,557:INFO:intervals after: [(22, 22)], [(278, 278)]
2023-11-03 08:52:30,558:INFO:evaluating the tp and tn solution for acc and spec
2023-11-03 08:52:30,559:INFO:intervals before: [(22, 22)], [(278, 278)]
2023-11-03 08:52:30,560:INFO:the tp solutions: (21.938733999999954, 2

False